In [1]:
#%pip uninstall Bio
#%conda install Bio

In [2]:
import pandas as pd
import io
import string
import numpy as np
import Bio
from Bio.Seq import Seq




In [3]:
df = pd.read_csv("barcodes_n9_k2.20230609_143112.csv")

In [4]:
df['barcode'] = df['barcode'].apply(lambda x: Bio.Seq.transcribe(x))
shuffled_df = df.sample(frac=1, random_state = 42).reset_index(drop = True)
shuffled_df

,barcode,gc,n,k,homopolymer,gc_min,gc_max
0,CCUUAUAGC,0.444444,9,2,4,0.3,0.7
1,CGACCCAUA,0.555556,9,2,4,0.3,0.7
2,ACAAAGUUG,0.333333,9,2,4,0.3,0.7
3,AGUCAACCC,0.555556,9,2,4,0.3,0.7
4,GAGACGAAG,0.555556,9,2,4,0.3,0.7
...,...,...,...,...,...,...,...
42612,AUACUAUGG,0.333333,9,2,4,0.3,0.7
42613,AGUGGUGUG,0.555556,9,2,4,0.3,0.7
42614,GUAUUUAGG,0.333333,9,2,4,0.3,0.7
42615,AACCAGCAG,0.555556,9,2,4,0.3,0.7


In [5]:
def hasCodon(text, codons):
  for i in range(0, len(text), 3):
    codon = text[i:i+3]
    if codon in codons:
       return True
  return False
codons = ['UAG', 'UGA', 'UAA','AUG', 'GUG', 'UUG']
trigger_barcodes = shuffled_df['barcode'].apply(lambda x: np.nan if hasCodon(x, codons) else x).dropna()

In [6]:
import random
from Bio.SeqUtils import gc_fraction
random.seed(420)
def random_dna_sequence(length):
    dna = ''.join(random.choice('ACUG') for _ in range(length))
    while(hasCodon(dna,codons) and gc_fraction(dna) > 0.6):
      dna = ''.join(random.choice('ACUG') for _ in range(length))
    return dna

shuffled_df['remaining_sequence'] = np.nan

shuffled_df['remaining_sequence'] = shuffled_df['remaining_sequence'].apply(lambda x: random_dna_sequence(6))
shuffled_df['a'] = shuffled_df['barcode'] + shuffled_df['remaining_sequence']
shuffled_df

,barcode,gc,n,k,homopolymer,gc_min,gc_max,remaining_sequence,a
0,CCUUAUAGC,0.444444,9,2,4,0.3,0.7,AUUGAA,CCUUAUAGCAUUGAA
1,CGACCCAUA,0.555556,9,2,4,0.3,0.7,AUGACU,CGACCCAUAAUGACU
2,ACAAAGUUG,0.333333,9,2,4,0.3,0.7,GUAUUC,ACAAAGUUGGUAUUC
3,AGUCAACCC,0.555556,9,2,4,0.3,0.7,AUGCGA,AGUCAACCCAUGCGA
4,GAGACGAAG,0.555556,9,2,4,0.3,0.7,CGAGAA,GAGACGAAGCGAGAA
...,...,...,...,...,...,...,...,...,...
42612,AUACUAUGG,0.333333,9,2,4,0.3,0.7,GGAAGU,AUACUAUGGGGAAGU
42613,AGUGGUGUG,0.555556,9,2,4,0.3,0.7,GCGCUC,AGUGGUGUGGCGCUC
42614,GUAUUUAGG,0.333333,9,2,4,0.3,0.7,AACGGA,GUAUUUAGGAACGGA
42615,AACCAGCAG,0.555556,9,2,4,0.3,0.7,UAACCC,AACCAGCAGUAACCC


In [7]:
shuffled_df['GC'] = shuffled_df['a'].apply(lambda x: gc_fraction(x))
shuffled_df = shuffled_df.query('GC < 0.6').reset_index(drop = True)
shuffled_df = shuffled_df.drop(['gc', 'n','k', 'homopolymer', 'gc_min', 'gc_max'], axis = 1)

In [8]:
print(shuffled_df)
a = shuffled_df.iloc[0:7538].sample(frac=1, random_state = 42).reset_index(drop = True)
b = shuffled_df.iloc[7539:15077].sample(frac=1, random_state = 42).reset_index(drop = True)
a.rename(columns={'barcode': 'a_barcode', 'remaining_sequence':'a_remaining_sequence'}, inplace=True)
b.rename(columns={'barcode': 'b_barcode', 'remaining_sequence':'b_remaining_sequence','a': 'b'}, inplace=True)
print(a)


         barcode remaining_sequence                a        GC
0      CCUUAUAGC             AUUGAA  CCUUAUAGCAUUGAA  0.500000
1      CGACCCAUA             AUGACU  CGACCCAUAAUGACU  0.583333
2      ACAAAGUUG             GUAUUC  ACAAAGUUGGUAUUC  0.500000
3      GAGACGAAG             CGAGAA  GAGACGAAGCGAGAA  0.533333
4      UUAACACAG             UACGGG  UUAACACAGUACGGG  0.583333
...          ...                ...              ...       ...
14970  UAAAGCUCA             GAUCUG  UAAAGCUCAGAUCUG  0.545455
14971  CCGCAAAGA             UCAGUU  CCGCAAAGAUCAGUU  0.583333
14972  AUACUAUGG             GGAAGU  AUACUAUGGGGAAGU  0.545455
14973  GUAUUUAGG             AACGGA  GUAUUUAGGAACGGA  0.545455
14974  AACCAGCAG             UAACCC  AACCAGCAGUAACCC  0.571429

[14975 rows x 4 columns]
      a_barcode a_remaining_sequence                a        GC
0     GUUACGGAG               AUAACC  GUUACGGAGAUAACC  0.583333
1     UGGUGAAGC               GAUAUA  UGGUGAAGCGAUAUA  0.545455
2     AGUGAGAUU           

In [9]:
targets = pd.merge(a, b, left_index=True, right_index=True)
targets['target']= targets['a'] + targets['b']
targets['GC_target'] = targets['target'].apply(lambda x: gc_fraction(x))
targets['GC_target'] = targets['GC_target'].apply(float)
targets['GC_target'] = targets['GC_target'].apply(lambda x: np.nan if x > 0.6 else x)
targets['target'] = targets['target'].apply(str)
targets['target'] = targets['target'].apply(lambda x: np.nan if hasCodon(x, codons) else x)
targets = targets[targets.notna().all(axis=1)]
targets['switch'] =  targets['target'].apply(lambda x: Bio.Seq.complement(x))
targets

c:\Users\ramul\.conda\envs\Bio\lib\site-packages\Bio\Seq.py:3200: BiopythonDeprecationWarning: complement(sequence) will change in the near future to always return DNA nucleotides only. Please use

complement_rna(sequence)

if you want to receive an RNA sequence instead.
  warnings.warn(


,a_barcode,a_remaining_sequence,a,GC_x,b_barcode,b_remaining_sequence,b,GC_y,target,GC_target,switch
2,AGUGAGAUU,UGCAUA,AGUGAGAUUUGCAUA,0.500000,CUUAACCCA,UACCGA,CUUAACCCAUACCGA,0.583333,AGUGAGAUUUGCAUACUUAACCCAUACCGA,0.545455,UCACUCUAAACGUAUGAAUUGGGUAUGGCU
10,AACAAGCCC,AGACCU,AACAAGCCCAGACCU,0.571429,GCUACACAC,AACUCA,GCUACACACAACUCA,0.538462,AACAAGCCCAGACCUGCUACACACAACUCA,0.555556,UUGUUCGGGUCUGGACGAUGUGUGUUGAGU
12,UAUGUUACC,UGGAAU,UAUGUUACCUGGAAU,0.555556,GUUGAAAUC,GCGAAA,GUUGAAAUCGCGAAA,0.500000,UAUGUUACCUGGAAUGUUGAAAUCGCGAAA,0.523810,AUACAAUGGACCUUACAACUUUAGCGCUUU
19,ACAUCCACG,ACAUGU,ACAUCCACGACAUGU,0.583333,CCAAUUAAG,CUUAGU,CCAAUUAAGCUUAGU,0.500000,ACAUCCACGACAUGUCCAAUUAAGCUUAGU,0.545455,UGUAGGUGCUGUACAGGUUAAUUCGAAUCA
22,CUAGCAGAG,GAUGAU,CUAGCAGAGGAUGAU,0.583333,AGAUUAGUC,AUUCGA,AGAUUAGUCAUUCGA,0.500000,CUAGCAGAGGAUGAUAGAUUAGUCAUUCGA,0.545455,GAUCGUCUCCUACUAUCUAAUCAGUAAGCU
...,...,...,...,...,...,...,...,...,...,...,...
7422,GAGCCUUCG,AAUCAA,GAGCCUUCGAAUCAA,0.583333,AGGGACUAC,UCCAUA,AGGGACUACUCCAUA,0.583333,GAGCCUUCGAAUCAAAGGGACUACUCCAUA,0.583333,CUCGGAAGCUUAGUUUCCCUGAUGAGGUAU
7423,CAACGUAAU,AACAAG,CAACGUAAUAACAAG,0.384615,GAACGAUGC,AUAUAU,GAACGAUGCAUAUAU,0.454545,CAACGUAAUAACAAGGAACGAUGCAUAUAU,0.416667,GUUGCAUUAUUGUUCCUUGCUACGUAUAUA
7424,CAUGUAAGA,CUCAUC,CAUGUAAGACUCAUC,0.545455,AUAAGUGCU,AGGACU,AUAAGUGCUAGGACU,0.545455,CAUGUAAGACUCAUCAUAAGUGCUAGGACU,0.545455,GUACAUUCUGAGUAGUAUUCACGAUCCUGA
7430,CCAUUCAAA,CGCAAU,CCAUUCAAACGCAAU,0.500000,UAUGAGCAA,GACGAU,UAUGAGCAAGACGAU,0.500000,CCAUUCAAACGCAAUUAUGAGCAAGACGAU,0.500000,GGUAAGUUUGCGUUAAUACUCGUUCUGCUA


In [10]:
def construct_fused(row):
    target = row['target']
    switch = row['switch']
    ascending_stem = row['ascending_stem']
    target_terminator = "AAATAATAAAAAAGCCGGATTAATAATCTGGCTTTTTATATTCTCT"
    target_promoter = "CGATCGTGACATGGATCCTGACTCGATCGGCAGTT"
    hairpin = "ACTGACTATTGCACAGAATAGTCAGT"
    spacer = "AATATTTCTAGATTTCAGTG"
    switch_promoter = "TTTACAGCTAGCTCAGTCCTAGGTATTATGCTAGC"
    nt7 = "CAGAAAC"
    RBS = "AGAGGAGA"
    start = "AUG"
    linker = "AACCTGGCGGCAGCGCAAAAG"
    return target_terminator + "CTA" + target + "TTT" + hairpin + "CCC" + target_promoter + spacer + switch_promoter + switch + ascending_stem + nt7 + RBS + Bio.Seq.complement(ascending_stem) + target[27:30] + start + target[23:24] + target[16:22] + linker
random.seed(420)
targets['ascending_stem'] = targets.apply(lambda x: random_dna_sequence(3), axis = 1)
targets= targets[targets.apply(lambda x: not hasCodon(x['ascending_stem'], codons) , axis=1)]
targets['fused_trigger_switch'] = targets.apply(lambda x: construct_fused(x), axis= 1)
targets = targets[targets.apply(lambda x: (not(x['a_barcode'] in x['fused_trigger_switch'][51:]) or (x['b_barcode'] in x['fused_trigger_switch'][65:])), axis=1)]
targets
targets.to_csv("FusedTargets.csv")                                          



c:\Users\ramul\.conda\envs\Bio\lib\site-packages\Bio\Seq.py:3200: BiopythonDeprecationWarning: complement(sequence) will change in the near future to always return DNA nucleotides only. Please use

complement_rna(sequence)

if you want to receive an RNA sequence instead.
  warnings.warn(
C:\Users\ramul\AppData\Local\Temp\ipykernel_11468\756970183.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  targets['fused_trigger_switch'] = targets.apply(lambda x: construct_fused(x), axis= 1)
